### Ingestion file Persom

In [0]:
dbutils.widgets.text("p_environment", "")
v_environment =  dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-30")
v_file_date =  dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, current_timestamp, concat, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.getOrCreate()

name_schema = StructType([
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True)
])

person_schema = StructType([
    StructField("personId", IntegerType(), False),
    StructField("personName", name_schema)
])



In [0]:
person = spark.read\
    .schema(person_schema)\
    .json(f"{bronze_folder_path}/{v_file_date}/person.json")

# Cambiar el nombre a las columnas, crear la nueva columna y eliminar la columna deseada
person_df = person\
    .withColumnRenamed("personId", "person_id")\
    .withColumn("ingestion_date", current_timestamp())\
    .withColumn("environment", lit("PROD"))\
    .withColumn("name", concat(col("personName.forename"),
                               lit(" "),
                               col("personName.surname")
                               )
                )\
    .withColumn("file_date", lit(v_file_date))\
    .drop(col('personName'))



In [0]:
merge_condition = 'tgt.person_id = src.person_id AND tgt.file_date = src.file_date'
merge_delta_lake(person_df, "movie_silver", "persons", silver_folder_path, merge_condition, "file_date")

# overwrite_partition(person_df, "movie_silver", "persons", "file_date")

In [0]:
# person_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.persons")


In [0]:
%sql
select file_date, count(1) 
from movie_silver.persons
group by file_date;

In [0]:
dbutils.notebook.exit("Success")